# DuckDB 向量相似性搜索 (VSS) 扩展教程 (使用 Sentence Transformers)

本教程演示了如何使用 DuckDB 的 `vss` 扩展进行向量相似性搜索，并结合 `sentence-transformers` 库（使用 `bge-m3` 模型）将文本转换为向量嵌入。

**先决条件:**
- Python 环境
- 安装了 `duckdb`, `sentence-transformers`, `numpy`, `pandas` 库。
- 下载了 `bge-m3` 模型并放置在指定路径 (例如 `C:\Users\k\Desktop\BaiduSyncdisk\baidu_sync_documents\hf_models\bge-m3`)。请根据您的实际路径修改代码中的 `model_path`。

In [1]:
# 安装必要的库 (如果尚未安装)
!pip install duckdb sentence-transformers numpy pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
import duckdb
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd # 用于更好地显示某些查询结果

# 初始化 DuckDB 连接 (此处使用内存数据库)
conn = duckdb.connect(':memory:')

# 安装并加载 vss 扩展
try:
    conn.execute("INSTALL vss;")
    conn.execute("LOAD vss;")
    print("VSS 扩展已成功安装并加载。")
except Exception as e:
    print(f"加载 VSS 扩展时出错: {e}\n可能已加载或安装路径存在问题。")

# 加载 sentence-transformer 模型
# 请确保将 model_path 替换为您的 bge-m3 模型实际路径
model_path = '/root/XiaokeAILabs/bge-m3'
try:
    model = SentenceTransformer(model_path)
    embedding_dim = model.get_sentence_embedding_dimension()
    print(f"SentenceTransformer 模型 (bge-m3) 已加载。嵌入维度: {embedding_dim}")
except Exception as e:
    print(f"加载 SentenceTransformer 模型失败: {e}\n请检查模型路径是否正确以及模型文件是否完整。")


VSS 扩展已成功安装并加载。
SentenceTransformer 模型 (bge-m3) 已加载。嵌入维度: 1024


## `vss` 扩展用法

`vss` 扩展是 DuckDB 的一个实验性扩展，它通过 DuckDB 的新固定大小 `ARRAY` 类型，为向量相似性搜索查询添加索引支持以加速查询。

### 创建 HNSW 索引

使用 `CREATE INDEX` 语句和 `USING HNSW` 子句，在具有 `ARRAY` 列的表上创建新的 HNSW (Hierarchical Navigable Small Worlds) 索引。
我们将创建一张表来存储文本及其通过 `bge-m3` 模型生成的向量嵌入。

In [15]:
if model: # 确保模型已加载
    conn.execute("DROP TABLE IF EXISTS text_embeddings_table;")
    conn.execute(f"CREATE TABLE text_embeddings_table (id INTEGER, sentence TEXT, embedding FLOAT[{embedding_dim}]);")

    sentences = [
        "今天天气真不错",
        "我喜欢吃苹果",
        "DuckDB 是一个分析型数据库",
        "向量搜索在人工智能中非常重要",
        "机器学习正在改变世界",
        "这是一个关于猫的文档",
        "那是一篇讨论狗的文章"
    ]

    for i, sentence in enumerate(sentences):
        embedding = model.encode(sentence)
        conn.execute("INSERT INTO text_embeddings_table VALUES (?, ?, ?)", [i, sentence, embedding.tolist()])

    conn.execute(f"CREATE INDEX text_hnsw_index ON text_embeddings_table USING HNSW (embedding);")
    print(f"已创建表 'text_embeddings_table' 并使用 {len(sentences)} 条文本嵌入构建 HNSW 索引 'text_hnsw_index'.")
else:
    print("模型未加载，跳过创建表和索引。")

ConnectionException: Connection Error: Connection already closed!

### 加速查询

该索引可用于加速使用 `ORDER BY` 子句评估支持的距离度量函数与索引列和常量向量之间的距离，并以 `LIMIT` 子句结尾的查询。

In [4]:
if model: # 确保模型已加载
    query_sentence = "关于数据库技术的讨论"
    query_embedding = model.encode(query_sentence)

    result_df = conn.execute(f"""
    SELECT id, sentence, array_distance(embedding, ?::FLOAT[{embedding_dim}]) as distance
    FROM text_embeddings_table
    ORDER BY distance
    LIMIT 3;
    """, [query_embedding.tolist()]).df()
    
    print(f"与 '{query_sentence}' 最相似的 3 个句子:")
    print(result_df)
else:
    print("模型未加载，跳过查询。")

与 '关于数据库技术的讨论' 最相似的 3 个句子:
   id          sentence  distance
0   2  DuckDB 是一个分析型数据库  0.912630
1   6        那是一篇讨论狗的文章  0.955702
2   3    向量搜索在人工智能中非常重要  1.006644


### 使用 `min_by` 函数

如果 `arg` 参数是匹配的距离度量函数，`min_by(col, arg, n)` 的重载版本也可以通过 `HNSW` 索引加速，可用于快速一次性最近邻搜索。

In [5]:
if model: # 确保模型已加载
    query_sentence_min_by = "人工智能的应用"
    query_embedding_min_by = model.encode(query_sentence_min_by)

    min_by_result_query = conn.execute(f"""
    SELECT
        min_by(
            text_embeddings_table, -- 要返回的表行
            array_distance(embedding, ?::FLOAT[{embedding_dim}]), -- 计算距离
            3  -- 返回前3个
        ) AS result
    FROM
        text_embeddings_table; -- 查询的数据表 (此处的 FROM 仅为满足 SQL 语法，实际操作对象是 min_by 的第一个参数)
    """, [query_embedding_min_by.tolist()]).fetchall()

    print(f"使用 min_by 查找与 '{query_sentence_min_by}' 最相似的 3 个结果:")
    if min_by_result_query and min_by_result_query[0] and min_by_result_query[0][0]:
        # min_by_result_query[0][0] 是一个列表，其中包含匹配行的字典
        for item in min_by_result_query[0][0]:
            # item 是一个字典，例如 {'id': ..., 'sentence': ..., 'embedding': ...}
            print(f"  ID: {item['id']}, Sentence: {item['sentence']}") # 为简洁起见，不打印 embedding
    else:
        print("No result from min_by query.")
else:
    print("模型未加载，跳过 min_by 查询。")

使用 min_by 查找与 '人工智能的应用' 最相似的 3 个结果:
  ID: 3, Sentence: 向量搜索在人工智能中非常重要
  ID: 4, Sentence: 机器学习正在改变世界
  ID: 5, Sentence: 这是一个关于猫的文档


### 验证索引使用

通过检查 `EXPLAIN` 输出并查找计划中的 `HNSW_INDEX_SCAN` 节点，可以验证索引是否被使用。

In [6]:
if model: # 确保模型已加载和 query_embedding 已定义 (来自前一个单元格)
    explain_output = conn.execute(f"""
    EXPLAIN
    SELECT id, sentence
    FROM text_embeddings_table
    ORDER BY array_distance(embedding, ?::FLOAT[{embedding_dim}])
    LIMIT 3;
    """, [query_embedding.tolist()]).fetchall()
    
    print("EXPLAIN 输出:")
    for row_type, plan_details in explain_output:
        print(f"--- {row_type} ---")
        print(plan_details)
else:
    print("模型未加载，跳过 EXPLAIN。")

EXPLAIN 输出:
--- physical_plan ---
┌───────────────────────────┐
│         PROJECTION        │
│    ────────────────────   │
│             #0            │
│             #1            │
│                           │
│          ~3 rows          │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│         PROJECTION        │
│    ────────────────────   │
│__internal_decompress_integ│
│     ral_integer(#0, 0)    │
│             #1            │
│             #2            │
│                           │
│          ~0 rows          │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│          ORDER_BY         │
│    ────────────────────   │
│           #2 ASC          │
└─────────────┬─────────────┘
┌─────────────┴─────────────┐
│         PROJECTION        │
│    ────────────────────   │
│__internal_compress_integra│
│     l_utinyint(#0, 0)     │
│             #1            │
│             #2            │
│                           │
│          ~0 rows          │
└─────

### 指定距离度量

默认情况下，HNSW 索引使用欧几里得距离 `l2sq` (L2 范数平方) 度量，与 DuckDB 的 `array_distance` 函数匹配，但也可以在创建索引时指定其他距离度量。

支持的距离度量及其对应的 DuckDB 函数如下表所示：
| Metric   | Function                     | Description      |
|----------|------------------------------|------------------|
| `l2sq`   | `array_distance`             | 欧几里得距离     |
| `cosine` | `array_cosine_distance`      | 余弦相似性距离   |
| `ip`     | `array_negative_inner_product` | 负内积           |

In [7]:
if model: # 确保模型已加载
    # 首先删除可能已存在的同名索引
    conn.execute("DROP INDEX IF EXISTS text_hnsw_cosine_index;")

    # 创建使用余弦相似性度量的 HNSW 索引
    conn.execute(f"""
    CREATE INDEX text_hnsw_cosine_index
    ON text_embeddings_table
    USING HNSW (embedding)
    WITH (
        metric = 'cosine'
    );
    """)
    print("已创建索引 'text_hnsw_cosine_index'，在 'text_embeddings_table' 上使用余弦相似度度量。")

    # 使用余弦距离进行查询示例
    query_sentence_cosine = "一个晴朗的日子"
    query_embedding_cosine = model.encode(query_sentence_cosine)

    result_cosine_df = conn.execute(f"""
    SELECT id, sentence, array_cosine_distance(embedding, ?::FLOAT[{embedding_dim}]) as cosine_distance
    FROM text_embeddings_table
    ORDER BY cosine_distance -- HNSW 索引将优化此 array_cosine_distance 调用
    LIMIT 3;
    """, [query_embedding_cosine.tolist()]).df()
    
    print(f"\n使用余弦相似度与 '{query_sentence_cosine}' 最相似的 3 个句子:")
    print(result_cosine_df)
else:
    print("模型未加载，跳过创建余弦索引和查询。")

已创建索引 'text_hnsw_cosine_index'，在 'text_embeddings_table' 上使用余弦相似度度量。

使用余弦相似度与 '一个晴朗的日子' 最相似的 3 个句子:
   id    sentence  cosine_distance
0   0     今天天气真不错         0.179287
1   1      我喜欢吃苹果         0.476196
2   5  这是一个关于猫的文档         0.510574


### 多索引支持

虽然每个 `HNSW` 索引仅适用于单个列，但可以在同一表上创建多个 `HNSW` 索引，每个索引分别索引不同的列。此外，也可以为同一列创建多个 `HNSW` 索引，每个索引支持不同的距离度量。

### Index Options

除了 `metric` 选项外，`HNSW` 索引创建语句还支持以下选项，用于控制索引构建和搜索过程的超参数：

| Option            | Default     | Description                                                                                                |
|-------------------|-------------|------------------------------------------------------------------------------------------------------------|
| `ef_construction` | 128         | 在构建索引时考虑的候选顶点数量。更高的值将使索引更准确，但也会增加构建索引所需的时间。                     |
| `ef_search`       | 64          | 在索引的搜索阶段考虑的候选顶点数量。更高的值将使索引更准确，但也会增加搜索所需的时间。                       |
| `M`               | 16          | 图中每个顶点保持的最大邻居数量。更高的值将使索引更准确，但也会增加构建索引所需的时间。                       |
| `M0`              | 2 * `M`     | 零级图中每个顶点保持的邻居数量。更高的值将使索引更准确，但也会增加构建索引所需的时间。                     |

此外，还可以在运行时通过设置 `SET hnsw_ef_search = <int>` 配置选项来覆盖在索引构建时设置的 `ef_search` 参数。如果希望在每个连接的基础上权衡搜索性能和准确性，这将非常有用。也可以通过调用 `RESET hnsw_ef_search` 来取消覆盖。

In [8]:
# 设置 hnsw_ef_search 参数
conn.execute("SET hnsw_ef_search = 100;")
current_ef_search = conn.execute("SELECT current_setting('hnsw_ef_search')").fetchone()[0]
print(f"hnsw_ef_search 设置为: {current_ef_search}")

# 重置 hnsw_ef_search 参数
conn.execute("RESET hnsw_ef_search;")
current_ef_search_after_reset = conn.execute("SELECT current_setting('hnsw_ef_search')").fetchone()
# 如果返回 None，表示它恢复到默认行为（可能由索引定义或编译时默认值决定）
print(f"hnsw_ef_search 已重置。当前值: {current_ef_search_after_reset[0] if current_ef_search_after_reset else '使用默认值'}")

hnsw_ef_search 设置为: 100
hnsw_ef_search 已重置。当前值: None


### 持久性 (Persistence)

由于一些已知的与自定义扩展索引持久化相关的问题，默认情况下，`HNSW` 索引只能在内存数据库的表上创建，除非将 `SET hnsw_enable_experimental_persistence = true` 配置选项设置为 `true`。

**警告**: 将此功能锁定在实验性标志后面的原因是 “WAL” 恢复尚未为自定义索引正确实现，这意味着如果在 `HNSW` 索引表上有未提交的更改时发生崩溃或数据库意外关闭，可能会导致**数据丢失或索引损坏**。不建议在生产环境中使用此功能。

In [9]:
# 启用 HNSW 索引的实验性持久化功能
conn.execute("SET hnsw_enable_experimental_persistence = true;")
persistence_status = conn.execute("SELECT current_setting('hnsw_enable_experimental_persistence')").fetchone()[0]
print(f"HNSW 索引实验性持久化功能已启用: {persistence_status}")

# 禁用 HNSW 索引持久化功能 (恢复默认)
conn.execute("SET hnsw_enable_experimental_persistence = false;")
persistence_status_reset = conn.execute("SELECT current_setting('hnsw_enable_experimental_persistence')").fetchone()[0]
print(f"HNSW 索引实验性持久化功能已重置为: {persistence_status_reset}")

HNSW 索引实验性持久化功能已启用: True
HNSW 索引实验性持久化功能已重置为: False


### 插入、更新、删除和重新压缩 (Inserts, Updates, Deletes and Re-Compaction)

HNSW 索引支持在创建索引后对表中的行进行插入、更新和删除。但是，需要注意以下两点：
- 在表中填充数据后创建索引会更快。
- 删除操作不会立即反映在索引中，而是被“标记”为已删除，这可能导致索引随时间变得陈旧。可以通过调用 `PRAGMA hnsw_compact_index('index_name')` 来触发索引的重新压缩。

In [10]:
if model: # 确保模型已加载
    conn.execute("DROP TABLE IF EXISTS compact_test_table;")
    conn.execute(f"CREATE TABLE compact_test_table (id INT, sentence TEXT, embedding FLOAT[{embedding_dim}]);")

    sentences_for_compact = [
        ("初始句子1", model.encode("初始句子1").tolist()),
        ("初始句子2", model.encode("初始句子2").tolist()),
        ("待删除句子", model.encode("待删除句子").tolist())
    ]

    for i, (sentence, embedding) in enumerate(sentences_for_compact):
        conn.execute("INSERT INTO compact_test_table VALUES (?, ?, ?)", [i, sentence, embedding])

    conn.execute(f"CREATE INDEX compact_hnsw_index ON compact_test_table USING HNSW (embedding);")
    print("已创建表 'compact_test_table' 和索引 'compact_hnsw_index'。")

    # 获取待删除句子的 ID
    id_to_delete_row = conn.execute("SELECT id FROM compact_test_table WHERE sentence = '待删除句子'").fetchone()
    if id_to_delete_row:
        id_to_delete = id_to_delete_row[0]
        conn.execute("DELETE FROM compact_test_table WHERE id = ?", [id_to_delete])
        print(f"已从 'compact_test_table' 表中删除 ID 为 {id_to_delete} (句子: '待删除句子') 的行。")
    else:
        print("未找到 '待删除句子'。")

    # 重新压缩索引
    conn.execute("PRAGMA hnsw_compact_index('compact_hnsw_index');")
    print("索引 'compact_hnsw_index' 已重新压缩。")

    # 验证 (可选)
    remaining_count = conn.execute("SELECT COUNT(*) FROM compact_test_table").fetchone()[0]
    print(f"重新压缩后 'compact_test_table' 中的行数: {remaining_count}")
else:
    print("模型未加载，跳过压缩示例。")

已创建表 'compact_test_table' 和索引 'compact_hnsw_index'。
已从 'compact_test_table' 表中删除 ID 为 2 (句子: '待删除句子') 的行。
索引 'compact_hnsw_index' 已重新压缩。
重新压缩后 'compact_test_table' 中的行数: 2


### 额外功能: 向量相似性搜索连接 (Vector Similarity Search Joins)

`vss` 扩展还提供了几个表宏，用于简化多个向量之间的匹配，即所谓的“模糊连接”。这些是：
- `vss_join(left_table, right_table, left_col, right_col, k, metric := 'l2sq')`
- `vss_match(right_table, left_col, right_col, k, metric := 'l2sq')`

这些函数**目前不使用 HNSW 索引**，但作为方便用户使用的实用工具函数提供。

In [11]:
if model: # 确保模型已加载
    conn.execute("DROP TABLE IF EXISTS text_haystack;")
    conn.execute("DROP TABLE IF EXISTS text_needle;")

    conn.execute(f"CREATE TABLE text_haystack (id INTEGER, sentence TEXT, embedding FLOAT[{embedding_dim}]);")
    conn.execute(f"CREATE TABLE text_needle (id INTEGER, query_sentence TEXT, query_embedding FLOAT[{embedding_dim}]);")

    haystack_sentences = [
        "这是一篇关于狗的文章",
        "猫是独立的宠物",
        "鸟儿在天空飞翔",
        "鱼儿在水里游泳",
        "人工智能的未来是光明的"
    ]
    for i, sentence in enumerate(haystack_sentences):
        embedding = model.encode(sentence).tolist()
        conn.execute("INSERT INTO text_haystack VALUES (?, ?, ?)", [i, sentence, embedding])

    needle_sentences = [
        "关于宠物的讨论",
        "科技发展趋势"
    ]
    for i, sentence in enumerate(needle_sentences):
        embedding = model.encode(sentence).tolist()
        conn.execute("INSERT INTO text_needle VALUES (?, ?, ?)", [i, sentence, embedding])

    print("已创建并填充表 'text_haystack' 和 'text_needle'。")

    # vss_join
    join_df = conn.execute(f"""
    SELECT
        res.left_tbl.query_sentence as query,
        res.right_tbl.sentence as match,
        res.score
    FROM
        vss_join(
            text_needle,       -- 左表
            text_haystack,     -- 右表
            query_embedding,   -- 左表中的向量列名
            embedding,         -- 右表中的向量列名
            2                  -- 为左表中的每个向量返回右表中最近的 k=2 个向量
        ) res;
    """).df()
    print("\nvss_join 连接结果:")
    print(join_df)

    # vss_match
    # vss_match 表现得像一个横向连接 (LATERAL JOIN)
    match_df = conn.execute(f"""
    SELECT
        N.query_sentence,
        matches.match_result -- vss_match 返回一个结构列表
    FROM
        text_needle N,
        vss_match(
            text_haystack,    -- 要搜索的表
            N.query_embedding, -- 来自左表的查询向量
            embedding,        -- text_haystack 中的向量列
            2                 -- k 个近邻
        ) AS matches(match_result);
    """).df()

    print("\nvss_match 函数执行结果:")
    # match_result 列包含一个匹配项列表，每个匹配项是一个字典
    for index, row in match_df.iterrows():
        print(f"\nQuery: {row['query_sentence']}")
        # Check if the list/array of matches is non-empty
        if len(row['match_result']) > 0:
            for match_item in row['match_result']: # match_item 是一个字典
                print(f"  Score: {match_item['score']:.4f}, Matched Sentence: {match_item['row']['sentence']}")
        else:
            print("  No matches found.")
else:
    print("模型未加载，跳过 vss_join 和 vss_match 示例。")

已创建并填充表 'text_haystack' 和 'text_needle'。

vss_join 连接结果:
     query        match     score
0   科技发展趋势  人工智能的未来是光明的  0.914081
1   科技发展趋势   这是一篇关于狗的文章  1.108213
2  关于宠物的讨论   这是一篇关于狗的文章  0.657268
3  关于宠物的讨论      猫是独立的宠物  0.844050

vss_match 函数执行结果:

Query: 科技发展趋势
  Score: 0.9141, Matched Sentence: 人工智能的未来是光明的
  Score: 1.1082, Matched Sentence: 这是一篇关于狗的文章

Query: 关于宠物的讨论
  Score: 0.6573, Matched Sentence: 这是一篇关于狗的文章
  Score: 0.8441, Matched Sentence: 猫是独立的宠物


### 限制 (Limitations)

- 目前仅支持由 `FLOAT` (32位，单精度) 组成的向量。
- 索引本身不进行缓冲管理，必须能够完全放入 RAM 内存中。
- 索引在内存中的大小不计入 DuckDB 的 `memory_limit` 配置参数。
- 除非将 `SET hnsw_enable_experimental_persistence = <bool>` 配置选项设置为 `true`，否则 `HNSW` 索引只能在内存数据库的表上创建 (参见“持久性”部分)。
- 向量连接表宏 (`vss_join` 和 `vss_match`) 不需要也不使用 `HNSW` 索引。

In [12]:
# 关闭 DuckDB 连接
conn.close()